In [91]:
from kyber_py.mlwe import MLWE
from kyber_py import PolynomialRing as pr
from kyber_py import Module as m
import os
import numpy as np
from pathlib import Path

def increase_byte(input_bytes, N):
    return (int.from_bytes(input_bytes, byteorder='big') + N).to_bytes(len(input_bytes), byteorder='big')

n = 50
q = 137
k = 1
logq = int(np.log2(q))
seed = 0

params = {
    'n': n,
    'q': q,
    'k': k,
    'seed': seed
}

mlwe = MLWE(params)
input_bytes = mlwe.get_random_bytes()


In [92]:
tiny_A = np.zeros((params["n"]*4*k, params["n"]*k))
for i in range(0, 4*params["n"]*k, k):
    increased_bytes = increase_byte(input_bytes, i)
    A = mlwe.generate_A(increased_bytes).to_list()
    tiny_A[i:i+k] = np.array(A).astype(int).reshape((k, params["n"]*k))

In [93]:
tiny_A

array([[ 73.,  96.,  66., ..., 114.,  50., 123.],
       [ 42.,  38.,  12., ..., 108., 106.,  88.],
       [ 27.,   3., 113., ...,  15.,  95.,  14.],
       ...,
       [ 75.,  23.,  25., ..., 113., 129.,  92.],
       [ 91.,  60., 126., ...,  95.,  42.,  11.],
       [ 21.,  33., 136., ...,  46.,  78.,  84.]], shape=(200, 50))

In [94]:
path = "/home/cristian/Documents/thesis/references/repos/LWE-benchmarking/data/benchmark_paper_data/n256_k2_logq12/origA_n256_k2_loq12.npy"

tiny_A = np.load(path, allow_pickle=True)
tiny_A

array([[ 294, 1765, 1659, ..., 2537, 1583, 1416],
       [3034, 2509, 2328, ..., 2279,  980, 1815],
       [1686, 1499,   77, ..., 3178,  733,  966],
       ...,
       [2580, 2533, 2800, ..., 1777, 2283,  348],
       [2296, 1418, 1547, ..., 1452, 1178, 2569],
       [ 889, 3045,  730, ...,  304, 2227,  753]], shape=(2048, 512))

In [10]:
from kyber_py.utils import find_root_of_unity, isprime, totient

for n in range(64, 65):
    for q in range(100000, 1000000):
        if isprime(q) and n <= totient(q):
            root = find_root_of_unity(n, q)
            if root != None and root < 10:
              print(n, q, root)

64 145601 6


KeyboardInterrupt: 

# Small Module-LWE

In [96]:
n = 8
q = 17
k = 2
logq = int(np.log2(q))
seed = 0

params = {
    'n': n,
    'q': q,
    'k': k,
    'seed': seed
}

mlwe = MLWE(params)
input_bytes = mlwe.get_random_bytes()


In [97]:
A = mlwe.generate_A(input_bytes)
A.reduce_symmetric_mod().to_list()

[[[1, -7, -5, -1, 2, -6, -7, 5], [-5, -4, -7, -1, -1, -7, -6, -5]],
 [[-5, -6, 2, -1, 4, -7, 1, 4], [8, 5, 3, 0, 3, -6, 1, 1]]]

In [98]:
s = mlwe.generate_secret(input_bytes)
s.reduce_symmetric_mod().to_list()

[[[-1, 1, 1, -1, 0, 1, 1, 0]], [[2, -1, 0, 0, -2, -2, -1, 1]]]

In [99]:
e = mlwe.generate_error(input_bytes)
e.reduce_symmetric_mod().to_list()

[[[0, 0, 0, 1, -1, 0, -1, -1]], [[-2, 1, 1, -2, -1, 0, 0, -1]]]

In [100]:
(A @ s).to_list()

[[[-4, -1, 6, -5, 1, -3, 8, -4]], [[-4, 5, 5, 0, -3, -8, 3, -3]]]

In [101]:
A_hat = A.to_ntt()
A_hat.to_list()

[[[8, 8, -2, 0, 1, -7, -3, 5], [-2, 0, 7, -5, -5, 7, -3, -1]],
 [[2, 7, -4, 1, -8, -4, 7, 6], [-2, 0, 7, -2, 7, -7, 3, -5]]]

In [102]:
s_hat = s.to_ntt()
s_hat.to_list()

[[[1, 1, -3, 3, -1, -1, -1, 1]], [[-1, -2, 1, -4, 5, 0, 3, 2]]]

In [103]:
e_hat = e.to_ntt()
e_hat.to_list()

[[[-2, 0, 0, 0, 2, 2, 0, -2]], [[-2, -2, -4, 4, 0, 0, -2, 2]]]

In [104]:
(A_hat @ s_hat).to_list()

[[[1, 3, -7, -5, -2, 7, 8, 0]], [[-6, -4, 8, 6, -4, -6, 6, -8]]]

# Size in bigger poly with hamming

In [105]:
n = 256
q = 3329
k = 2
logq = int(np.log2(q))
seed = 1

params = {
    'n': n,
    'q': q,
    'k': k,
    'seed': seed
}

mlwe = MLWE(params)
input_bytes = mlwe.get_random_bytes()

In [106]:
max_hamming = 10

A = mlwe.generate_A(input_bytes)
s = mlwe.generate_secret(input_bytes, max_hamming)
e = mlwe.generate_error(input_bytes, max_hamming)
print("Size of A:", A.size())
print("Size of s:", s.size())
print("Size of e:", e.size())

A_hat = A.to_ntt()
s_hat = s.to_ntt()
e_hat = e.to_ntt()
print("Size of A_hat:", A_hat.size())
print("Size of s_hat:", s_hat.size())
print("Size of e_hat:", e_hat.size())

Size of A: 1664
Size of s: 2
Size of e: 2
Size of A_hat: 1663
Size of s_hat: 1663
Size of e_hat: 1663


In [114]:
B = A @ s + e

B_hat = B.to_ntt()
m.distance((B_hat - e_hat).from_ntt(), B - e)

0

In [70]:
from kyber_py import Module

m = Module(2, 11)

A_00 = np.array([1, 0])
A_01 = np.array([0, 0])
A_10 = np.array([0, 0])
A_11 = np.array([0, 0])

S_00 = np.array([1, 1])
S_10 = np.array([0, 0])

A = m.from_list([   [ A_00.tolist(), A_01.tolist()  ],
                    [ A_10.tolist(), A_11.tolist()  ]   ])

s = m.from_list([   [ S_00.tolist()  ],
                    [ S_10.tolist()  ]   ])

(A @ s)

[1 + x]
[    0]

In [67]:
A_00 * S_00 + A_01 * S_10

array([1, 0])